<a href="https://colab.research.google.com/github/mnijhuis-dnb/open_source_workshop/blob/master/CaseStudySolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

def download_files(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
file_Global_mobility_report = '/Global_mobility_report.csv'
file_PPP_Loan_data = '/PPP_Loan_data.csv'
file_US_zips = '/uszips.xslx'

# download_files('1Fe73x8Splw9xY5FOTnpLWENJaBU2tHEy', file_NAICS)
download_files('1d4C6sTuD53tc_Ewbfb1iXascKzDSzDl0', file_Global_mobility_report)
# download_files('1KzT1XNA9K0enUv73nwPSkBYp7btg0pvJ', file_US_Hours_Worked)
download_files('1_7s1tH7FVNA5s5gu1vcWxikmGYQ2ryvn', file_PPP_Loan_data)
download_files('1f-ru7e4Ml0d5uzOMGtNpmOQ3kGSr7iT_', file_US_zips)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_zips = pd.read_excel(file_US_zips)
df_zips = df_zips[['zip','county_fips','population']]

In [ ]:
df_mob = pd.read_csv(file_Global_mobility_report)
df_mob = df_mob.loc[df_mob['country_region_code']=='US']
df_mob.drop(columns=['country_region_code','country_region','iso_3166_2_code'], inplace=True)
df_mob.dropna(subset=['census_fips_code'], inplace= True)
#df_mob = pd.merge(df_mob, df_zips, how='inner', left_on='census_fips_code', right_on='county_fips')
df_mob.head()

In [ ]:
df_PPP = pd.read_csv(file_PPP_Loan_data, low_memory=False)
df_PPP.drop(columns=['Unnamed: 0', 'BusinessType', 'Address', 'BusinessName', 'CD', 'City', 'DateApproved', 'Lender', 'State', 'Gender', 'NonProfit', 'Veteran','RaceEthnicity'], inplace=True)
df_PPP['LoanAmount'] = pd.to_numeric(df_PPP['LoanAmount'], errors='coerce')
df_PPP['JobsRetained'] = pd.to_numeric(df_PPP['JobsRetained'], errors='coerce')

In [ ]:
print(df_PPP['LoanRange'].unique())

In [ ]:
df_PPP.loc[df_PPP['LoanRange']=='a $5-10 million','LoanRange'] = 7000000
df_PPP.loc[df_PPP['LoanRange']=='b $2-5 million','LoanRange'] = 3000000
df_PPP.loc[df_PPP['LoanRange']=='c $1-2 million','LoanRange'] = 1500000
df_PPP.loc[df_PPP['LoanRange']=='d $350,000-1 million','LoanRange'] = 700000
df_PPP.loc[df_PPP['LoanRange']=='e $150,000-350,000','LoanRange'] = 250000
df_PPP['LoanRange'] = pd.to_numeric(df_PPP['LoanRange'], errors='coerce')

In [ ]:
df_PPP['LoanRange'].fillna(0,inplace=True)
df_PPP['LoanAmount'].fillna(0,inplace=True)
df_PPP['LoanAmount'] = df_PPP['LoanAmount'] + df_PPP['LoanRange']
df_PPP.drop(columns='LoanRange',inplace=True)

In [ ]:
df_mob['change_neg'] = df_mob[['retail_and_recreation_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline', 'workplaces_percent_change_from_baseline']].mean(axis=1)
df_mob['change_pos'] = df_mob[['parks_percent_change_from_baseline', 'residential_percent_change_from_baseline']].mean(axis=1)
#df_mob.drop(columns=df_mob.columns[5:-1], inplace=True)
df_mob[['metro_area', 'sub_region_2']] = df_mob[['metro_area', 'sub_region_2']].fillna('-')
df_mob = df_mob.groupby(by=['census_fips_code', 'metro_area', 'sub_region_1', 'sub_region_2']).mean().reset_index()

In [ ]:
df_PPP = pd.pivot_table(df_PPP, values=['JobsRetained','LoanAmount'], index='Zip').reset_index()

In [ ]:
df_combined = pd.merge(df_PPP, df_zips, how='left', left_on='Zip', right_on='zip').drop(columns=['zip','Zip'])
df_combined = pd.pivot_table(df_combined, values=['JobsRetained','LoanAmount','population'], index='county_fips', aggfunc='sum').reset_index()
df_combined = pd.merge(df_combined, df_mob, how='left', left_on='county_fips', right_on='census_fips_code').drop(columns=['county_fips','census_fips_code'])
df_combined.head(10)

In [ ]:
df_combined['JobsRetained'] = df_combined['JobsRetained']/df_combined['population']*1000
df_combined['LoanAmount'] = df_combined['LoanAmount']/df_combined['population']
df_combined = df_combined.loc[df_combined['population'] > 10000]

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 9]
plot = df_combined.loc[df_combined['LoanAmount']<20].plot.scatter(x='LoanAmount', y='retail_and_recreation_percent_change_from_baseline', alpha=0.2)
df_combined.loc[df_combined['LoanAmount']<20].plot.scatter(x='LoanAmount', y='transit_stations_percent_change_from_baseline', alpha=0.2)
df_combined.loc[df_combined['LoanAmount']<20].plot.scatter(x='LoanAmount', y='workplaces_percent_change_from_baseline', alpha=0.2)
df_combined.loc[df_combined['LoanAmount']<20].plot.scatter(x='LoanAmount', y='grocery_and_pharmacy_percent_change_from_baseline', alpha=0.2)
df_combined.loc[df_combined['LoanAmount']<20].plot.scatter(x='LoanAmount', y='residential_percent_change_from_baseline', alpha=0.2)
df_combined.loc[df_combined['LoanAmount']<20].plot.scatter(x='LoanAmount', y='parks_percent_change_from_baseline', alpha=0.2)
df_combined.loc[df_combined['LoanAmount']<20].plot.scatter(x='LoanAmount', y='change_pos', alpha=0.2)
df_combined.loc[df_combined['LoanAmount']<20].plot.scatter(x='LoanAmount', y='change_neg', alpha=0.2)

In [ ]:
df_combined.corr()

In [ ]:
col_combined = df_combined.columns[6:]
df_combined.sort_values(by=['residential_percent_change_from_baseline'] , ascending=False, na_position='last').drop(columns=col_combined.drop('residential_percent_change_from_baseline')).head(10)

In [ ]:
df_combined.sort_values(by=['workplaces_percent_change_from_baseline'] , ascending=True, na_position='last').drop(columns=col_combined.drop('workplaces_percent_change_from_baseline')).head(10)